In [48]:
from pathlib import Path
from app.config import DATASET_PATH, MODEL_SOURCE
from app.utils import load_dataset, load_model_and_tokenizer, preprocess
from scipy.special import softmax


In [49]:
tokenizer, model = load_model_and_tokenizer(MODEL_SOURCE)


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
dataset = load_dataset(DATASET_PATH).shuffle()["test"][:N_SAMPLES]


In [33]:
import torch

In [47]:
N_BEVAL_BATCH_SIZE = 64
N_SAMPLES = 500
N_BATCHES = len(dataset["text"])//EVAL_BATCH_SIZE
N_BATCHES

2

In [54]:
EVAL_BATCH_SIZE = 64
N_SAMPLES = 500
def evaluate_accuracy():

    dataset = load_dataset(DATASET_PATH).shuffle()["test"][:N_SAMPLES]
    N_BATCHES = len(dataset["text"])//EVAL_BATCH_SIZE

    accuracy = 0
    for i in range(N_BATCHES+1):

        start = i*EVAL_BATCH_SIZE
        end = min(N_SAMPLES, (i+1)*EVAL_BATCH_SIZE)
        print(start, end)
        samples, labels = dataset["text"][start:end], dataset["label"][start:end]
        
        model.eval()
        encoded_batch = tokenizer(
            [preprocess(t) for t in samples],
            padding=True,          # pad to same length
            truncation=True,       # truncate long texts
            return_tensors="pt",
        )

        with torch.no_grad():
            output = model(**encoded_batch)
    
        logits = output[0].detach().cpu().numpy()
        scores = softmax(logits, axis=-1)
        pred_labels = scores.argmax(axis=-1)
        accuracy += sum(pred_labels==labels)
    accuracy/=N_SAMPLES
    return accuracy
evaluate_accuracy()

0 64
64 128
128 192
192 256
256 320
320 384
384 448
448 500


np.float64(0.71)

In [1]:
def _load_test_data():
    """
    Expects CSV with columns: text,label
    label values must be one of labels (negative, neutral, positive) or their indices (0,1,2).
    """
    df = pd.read_csv(TEST_DATA_PATH)
    # normalize label column to strings matching our 'labels' list
    if np.issubdtype(df["label"].dtype, np.number):
        df["label"] = df["label"].astype(int).map(lambda i: labels[i])
    else:
        df["label"] = df["label"].str.lower().str.strip()
    # keep only supported labels
    df = df[df["label"].isin(labels)].dropna(subset=["text", "label"])
    return df